<a href="https://colab.research.google.com/github/piranfar/docking-Analyze/blob/main/Dockinganalyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from google.colab import files

def analyze_dlg():
    """Analyzes AutoDock DLG file, extracts binding energies,
    creates a table and plot, and saves results."""

    # Step 1: Upload the AutoDock DLG file
    print("📤 Please upload your AutoDock results file (.dlg):")
    uploaded = files.upload()

    # Get the filename
    dlg_filename = list(uploaded.keys())[0]

    # Step 2: Extract Binding Energies with Regex
    binding_energies = []
    pattern = r"Estimated Free Energy of Binding\s+=\s+(-?\d+\.\d+)\s+kcal/mol"

    with open(dlg_filename, "r", encoding="utf-8", errors="ignore") as file:
        for line in file:
            match = re.search(pattern, line)
            if match:
                binding_energies.append(float(match.group(1)))

    # Step 3: Store results in a DataFrame
    df_binding_energies = pd.DataFrame({
        "Pose": list(range(1, len(binding_energies) + 1)),
        "Binding Energy (kcal/mol)": binding_energies
    })

    # Step 4: Display the extracted data
    if df_binding_energies.empty:
        print("⚠️ No binding energies were extracted. Please check your DLG file format.")
        return  # Exit if no data found

    from IPython.display import display
    display(df_binding_energies)

    # Step 5: Save results as CSV
    csv_filename = "binding_energies.csv"
    df_binding_energies.to_csv(csv_filename, index=False)
    print(f"✅ Results saved as '{csv_filename}'. You can download it below:")
    files.download(csv_filename)

    # Step 6: Visualization
    def _plot_series(series, series_name, series_index=0):
        palette = list(sns.palettes.mpl_palette('Dark2'))
        xs = series['Pose']
        ys = series['Binding Energy (kcal/mol)']
        plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

    fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
    df_sorted = df_binding_energies.sort_values('Pose', ascending=True)
    _plot_series(df_sorted, '')
    sns.despine(fig=fig, ax=ax)
    plt.xlabel('Pose')
    plt.ylabel('Binding Energy (kcal/mol)')
    plt.title("Pose vs Binding Energy (kcal/mol)")  # Add title
    plt.show()

# Call the function to start the analysis
analyze_dlg()